In [1]:
import time
from array import array
from random import sample, shuffle, randint

In [2]:
genome_length = 100
test_count = 1000000
alternations = 4

# Helper Functions

In [3]:
def stopwatch(count, method, *args):
    start_time = time.time()
    for _ in range(count):
        method(*args)
    return time.time() - start_time

def alternate(count, method1, args1, method2, args2):
    methods = [method1, method2]
    args = [args1, args2]
    output = [[], []]
    for i in range(count*2):
        output[i%2].append(methods[i%2](*args[i%2]))
    return output[0], output[1]

def sums(*args):
    results = []
    for arg in args:
        total = 0
        for element in arg:
            total += element
        results.append(total)
    return tuple(results)
    
def lst_individual(genome_length):
    return sample(list(range(genome_length)), genome_length)

def arr_individual(genome_length):
    return array('i', lst_individual(genome_length))

def print_times(string1, time1, string2, time2):
    print("%s: %.2f ms\n%s: %.2f ms\nPerformance Difference: %.2f%%" % (string1, time1*1000, string2, time2*1000, ((time1/time2)-1)*100))

def compare(count, alternations, method1, args1, method2, args2):
    return sums(*alternate(alternations, stopwatch, [count//alternations, method1, *args1], stopwatch, [count//alternations, method2, *args2]))

def gen_two_nums():
    x = randint(0, genome_length - 1)
    y = randint(0, genome_length - 1)
    return x, y

def gen_two_nums_ascending():
    # Generate two integers such that x > y
    x = randint(0, genome_length - 2)
    y = randint(x+1, genome_length - 1)
    return x, y

# Mutation Methods

## Swap

In [4]:
def permutation_swap(individual):
    # Generate two random indices
    x, y = gen_two_nums()

    # Swap the values at those indices
    individual[x], individual[y] = individual[y], individual[x]

    return individual

In [5]:
times = compare(test_count, alternations,
                permutation_swap, [lst_individual(genome_length)],
                permutation_swap, [arr_individual(genome_length)]
               )

print_times("List", times[0], "Array", times[1])

List: 231.61 ms
Array: 259.43 ms
Performance Difference: -10.73%


## Insert

In [6]:
def permutation_insert(individual):
    # Generate two random indices
    x, y = gen_two_nums()

    # Insert the value at y in the position after x
    value = individual.pop(y)
    individual.insert(x+1, value)

    return individual

In [7]:
times = compare(test_count, alternations,
                permutation_insert, [lst_individual(genome_length)],
                permutation_insert, [arr_individual(genome_length)]
               )

print_times("List", times[0], "Array", times[1])

List: 341.73 ms
Array: 288.38 ms
Performance Difference: 18.50%


## Inversion

In [50]:
def permutation_inversion_lst(individual):
    # Generate two random indices in ascending order
    x, y = gen_two_nums_ascending()

    # Reverse the contents from x to y
    individual[x:y] = individual[x:y][::-1]
    return individual

def permutation_inversion_arr(individual):
    x, y = gen_two_nums_ascending()
    for i in range((y-x)//2):
        individual[x+i], individual[y-i] = individual[y-i], individual[x+i]
            

In [70]:
times = compare(test_count, alternations,
                permutation_inversion_lst, [lst_individual(genome_length)],
                permutation_inversion_arr, [arr_individual(genome_length)]
               )

print_times("List", times[0], "Array", times[1])

List: 360.03 ms
Array: 337.68 ms
Performance Difference: 6.62%


## Scramble

In [10]:
def permutation_scramble(individual):
    # Generate two random indices in ascending order
    x, y = gen_two_nums_ascending()

    # Randomize the order of indices from x to y
    temp = individual[x:y]
    shuffle(temp)
    individual[x:y] = temp

    return individual

In [76]:
times = compare(test_count, alternations,
                permutation_scramble, [lst_individual(genome_length)],
                permutation_scramble, [arr_individual(genome_length)]
               )

print_times("List", times[0], "Arr", times[1])

List: 3955.23 ms
Arr: 3523.30 ms
Performance Difference: 12.26%


# Recombination Methods

## Order Crossover

In [75]:
def order_crossover_arr(parent1, parent2):
    # Makes the offspring from the selected sub-sequence, and all the elements not in that sub-sequence.
    offspring1 = parent1[:p1] + array('i', [x for x in parent2[p1:] + parent2[:p1] if x not in set(parent1[:p1])])
    offspring2 = parent2[:p1] + array('i', [x for x in parent1[p1:] + parent1[:p1] if x not in set(parent2[:p1])])
    return offspring1, offspring2

def order_crossover_lst(parent1, parent2):
    # Makes the offspring from the selected sub-sequence, and all the elements not in that sub-sequence.
    offspring1 = parent1[:p1] + [x for x in parent2[p1:] + parent2[:p1] if x not in set(parent1[:p1])]
    offspring2 = parent2[:p1] + [x for x in parent1[p1:] + parent1[:p1] if x not in set(parent2[:p1])]
    return offspring1, offspring2

In [78]:
times = compare(test_count, alternations,
                order_crossover_lst, [lst_individual(genome_length), lst_individual(genome_length)],
                order_crossover_arr, [arr_individual(genome_length), arr_individual(genome_length)]
               )

print_times("List", times[0], "Array", times[1])

NameError: name 'p1' is not defined